In [100]:
from bs4 import BeautifulSoup
import re
import bisect
import pandas as pd

In [3]:
with open('data/foia 14-3668 officers with cr allegations-html.html', 'r') as f:
    foia_html = f.read()
complaint_soup = BeautifulSoup(foia_html, 'html.parser')

In [120]:
bolds = complaint_soup.findAll("b")
len(bolds)

332656

In [139]:
bold_styles = [re.search('left:\d{1,3}px', b.parent.get('style', 'left:00px')) for b in bolds]
bold_style_list = [b.group(0) for b in bold_styles if b is not None]
len(bold_style_list)

332655

In [140]:
bold_style_list[:5]

['left:41px', 'left:566px', 'left:507px', 'left:569px', 'left:523px']

In [141]:
bold_style_set = set(bold_style_list)
len(bold_style_set)

15

In [142]:
bold_style_set

{'left:00px',
 'left:179px',
 'left:182px',
 'left:232px',
 'left:39px',
 'left:41px',
 'left:42px',
 'left:487px',
 'left:507px',
 'left:50px',
 'left:523px',
 'left:553px',
 'left:566px',
 'left:569px',
 'left:71px'}

In [173]:
all_ps = complaint_soup.findAll("p")
p_styles = [re.search('left:\d{1,3}px', p.get('style', 'left:00px')) for p in all_ps]
p_style_list = [p.group(0) for p in p_styles if p is not None]
len(p_style_list)

1658703

In [205]:
p_action_items = [p for p in all_ps if re.match('.*left:850px.*', p['style'])]
print(len(p_action_items))
p_actions = [p for p in p_action_items if re.match('\d{2,4}.*', p.get_text())]
print(len(p_actions))

113681
113681


In [208]:
p_finding_items = [p for p in all_ps if re.match('.*left:78[1567].*', p['style'])]
print(len(p_finding_items))
p_findings = [p for p in p_finding_items if re.match('[A-Z]{2}', p.get_text())]
print(len(p_findings))

144313
113637


In [199]:
p_closed_items = [p for p in all_ps if re.match('.*left:69[35678]px.*', p['style'])]
print(len(p_696_items))
p_closed_dates = [p for p in p_696_items if re.match('\d{2}[ -]\w{2,4}[ -]\d{4}', p.get_text())]
print(len(p_696_dates))

133123
117785


In [200]:
p_complaint_dates = [p for p in all_ps if re.match('.*left:61[01234]px.*', p['style'])]
print(len(p_complaint_dates))
p_complaint_date_items = [p for p in p_complaint_dates if re.match('\d{2}[ -]\w{2,4}[ -]\d{4}', p.get_text())]
print(len(p_complaint_date_items))

133252
117914


In [201]:
p_incident_dates = [p for p in all_ps if re.match('.*left:54[01234567]px.*', p['style'])]
print(len(p_incident_dates))
p_incident_date_items = [p for p in p_incident_dates if re.match('\d{2}[ -]\w{2,4}[ -]\d{4}', p.get_text())]
print(len(p_incident_date_items))

133252
117914


In [220]:
p_complaint_categories = [p for p in all_ps if re.match('.*left:173px.*', p['style'])]
print(len(p_complaint_categories))

235828


In [175]:
p_style_set = set(p_style_list)
len(p_style_set)

49

In [176]:
# Show unique combinations of locations to find what should search for
p_style_set

{'left:126px',
 'left:134px',
 'left:169px',
 'left:173px',
 'left:179px',
 'left:182px',
 'left:232px',
 'left:249px',
 'left:346px',
 'left:39px',
 'left:400px',
 'left:41px',
 'left:42px',
 'left:487px',
 'left:507px',
 'left:50px',
 'left:523px',
 'left:540px',
 'left:541px',
 'left:542px',
 'left:543px',
 'left:547px',
 'left:553px',
 'left:556px',
 'left:566px',
 'left:569px',
 'left:610px',
 'left:611px',
 'left:612px',
 'left:613px',
 'left:614px',
 'left:627px',
 'left:693px',
 'left:695px',
 'left:696px',
 'left:697px',
 'left:698px',
 'left:71px',
 'left:781px',
 'left:785px',
 'left:786px',
 'left:787px',
 'left:83px',
 'left:850px',
 'left:85px',
 'left:88px',
 'left:915px',
 'left:917px',
 'left:94px'}

### Notes

* Each column has unique left margin (with exceptions such as the `***` for repeated items), so take that to create columns
* Each one also has a height attribute, can use that to group them by officer
* First get list of officers, then list of complain number rows
* Use bisection for linear search, sort every item by a key that is the page multiplied by 1000 plus the location within the page div
* Can use the previous index given by bisection to place officers with complaints

#### Row Details
* Complaint number can be at slightly different location than actual row, subtract 3 px from it to make sure that it is slightly less than its associated row
* Secondary (final code row) can just use violation code, only use items from the code column, and use bisection, but check if it is more than 5 pixels greater in location than its associated complaint. If so, then assign it to the final code attribute 

In [7]:
officers = complaint_soup.findAll("p", {"style" : re.compile('left:41px*')})
len(officers)

15339

In [154]:
def get_p_loc(p, id_string):
    p_dict = dict()
    p_dict[id_string] = p.get_text()
    p_dict['page'] = int(p['class'][0][2:-1])
    p_dict['location'] = int(re.compile('top:(\d{1,6})px').search(str(p)).group(1))
    p_dict['key'] = (p_dict['page']*1000) + p_dict['location']
    return p_dict

In [246]:
officer_list = list()
for officer in officers:
    officer_list.append(get_p_loc(officer, 'officer_name'))
officer_list[:5]

[{'key': 1238,
  'location': 238,
  'officer_name': 'PIWNICKI, RAYMOND',
  'page': 1},
 {'key': 5633, 'location': 633, 'officer_name': 'FINNIGAN, JEROME', 'page': 5},
 {'key': 10174, 'location': 174, 'officer_name': 'CAMPBELL, SEAN', 'page': 10},
 {'key': 14341,
  'location': 341,
  'officer_name': 'WOJTAN, KENNETH',
  'page': 14},
 {'key': 18341, 'location': 341, 'officer_name': 'HERRERA, KEITH', 'page': 18}]

In [247]:
officer_list = sorted(officer_list, key=lambda k: k['key'])
officer_keys = [officer['key'] for officer in officer_list]
print(len(officer_list))
print(len(officer_keys))

15338
15338


In [248]:
p_appt_date = [p for p in all_ps if re.match('.*left:346px.*', p['style'])]
print(len(p_appt_date))
p_appt_date_list = [p for p in p_appt_date if re.match('\d{2}[ -]\w{2,4}[ -]\d{4}', p.get_text())]
print(len(p_appt_date_list))
appt_date_list = list()
for ad in p_appt_date_list:
    appt_date_list.append(get_p_loc(ad, 'appt_date'))
print(len(appt_date_list))

15338
15338
15338


In [249]:
# Combine appt_date and officers, must be done before merge, some with same name
for ad in appt_date_list:
    # Add additional to make sure because not worried about being too close together
    off_idx = bisect.bisect(officer_keys, ad['key'] + 10)
    officer_list[off_idx-1]['appt_date'] = ad['appt_date']
len(officer_list)

15338

In [143]:
cr_nos = complaint_soup.findAll("p", {"style" : re.compile('left:42px*|left:41px*|left:39px*|left:50px*')})
len(cr_nos)

133253

In [152]:
cr_list = list()
for cr in cr_nos:
    if re.match('\d{4,9}|\*\*\*', cr.get_text()) is not None:
        cr_list.append(get_p_loc(cr, 'cr_no'))
print(len(cr_list))
cr_list[:5]

117914


[{'cr_no': '263295', 'key': 1273, 'location': 273, 'page': 1},
 {'cr_no': '263967', 'key': 1314, 'location': 314, 'page': 1},
 {'cr_no': '265117', 'key': 1356, 'location': 356, 'page': 1},
 {'cr_no': '266694', 'key': 1398, 'location': 398, 'page': 1},
 {'cr_no': '266986', 'key': 1440, 'location': 440, 'page': 1}]

In [250]:
#sorted_cr_list = sorted(cr_list, key=lambda k: k['key']) 
sorted_officer_list = sorted(officer_list, key=lambda k: k['key'])
sorted_cr_list[:5]

[{'cr_no': '263295', 'key': 1273, 'location': 273, 'page': 1},
 {'cr_no': '263967', 'key': 1314, 'location': 314, 'page': 1},
 {'cr_no': '265117', 'key': 1356, 'location': 356, 'page': 1},
 {'cr_no': '266694', 'key': 1398, 'location': 398, 'page': 1},
 {'cr_no': '266986', 'key': 1440, 'location': 440, 'page': 1}]

In [252]:
cr_officer_merge = list()
for cr in cr_list:
    off_idx = bisect.bisect(officer_keys, cr['key'])
    cr_item = cr.copy()
    # Add officer before insertion point
    cr_item['officer_name'] = officer_list[off_idx-1]['officer_name']
    cr_item['appt_date'] = officer_list[off_idx-1]['appt_date']
    cr_officer_merge.append(cr_item)
len(cr_officer_merge)

117914

In [253]:
cr_df = pd.DataFrame(cr_officer_merge)
cr_df.head()

,appt_date,cr_no,key,location,officer_name,page
0,08-Jun-1998,263295,1273,273,"PIWNICKI, RAYMOND",1
1,08-Jun-1998,263967,1314,314,"PIWNICKI, RAYMOND",1
2,08-Jun-1998,265117,1356,356,"PIWNICKI, RAYMOND",1
3,08-Jun-1998,266694,1398,398,"PIWNICKI, RAYMOND",1
4,08-Jun-1998,266986,1440,440,"PIWNICKI, RAYMOND",1


In [254]:
cr_df.to_csv('initial_cr_officer_merge.csv',index=False)

In [257]:
cr_codes = complaint_soup.findAll("p", {'style': re.compile('left:134px*')})
len(cr_codes)
# Matches up exactly to double the number of complaints

235828

In [165]:
cr_code_list = list()
for cr in cr_codes:
    cr_code_list.append(get_p_loc(cr, 'cr_code'))
len(cr_code_list)

235828

In [255]:
complaint_keys = [c['key'] for c in cr_officer_merge]
complaint_keys[:5]

[1273, 1314, 1356, 1398, 1440]

In [258]:
# Because complaint top location may be slightly lower than the cr code, bisect with a 3px padding added
# if the difference is greater than 5 px between the heights, add as the final code
for cr in cr_code_list:
    cr_idx = bisect.bisect(complaint_keys, (cr['key'] + 3))
    # Check if the difference between the code and the complaint row is more than 5 px, if so, add as final
    if ((cr['key'] + 3) - cr_officer_merge[cr_idx-1]['key']) > 5:
        cr_officer_merge[cr_idx-1]['final_category_code'] = cr['cr_code']
    else:
        cr_officer_merge[cr_idx-1]['initial_category_code'] = cr['cr_code']
len(cr_officer_merge)

117914

In [260]:
cr_df = pd.DataFrame(cr_officer_merge)
cr_df.to_csv('cr_officer_merge_codes.csv',index=False)

In [211]:
cr_action_list = list()
for cr in p_actions:
    cr_action_list.append(get_p_loc(cr, 'action'))
print('Action list length : {}'.format(len(cr_action_list)))

cr_finding_list = list()
for cr in p_findings:
    cr_finding_list.append(get_p_loc(cr, 'finding'))
print('Finding list length : {}'.format(len(cr_finding_list)))

closed_date_list = list()
for cr in p_closed_dates:
    closed_date_list.append(get_p_loc(cr, 'closed_date'))
print('Closed date list length: {}'.format(len(closed_date_list)))

incident_date_list = list()
for cr in p_incident_date_items:
    incident_date_list.append(get_p_loc(cr, 'incident_date'))
print('Incident date list length: {}'.format(len(incident_date_list)))

complaint_date_list = list()
for cr in p_complaint_date_items:
    complaint_date_list.append(get_p_loc(cr, 'complaint_date'))
print('Complaint date list length: {}'.format(len(complaint_date_list)))

Action list length : 113681
Finding list length : 113637
Closed date list length: 117785
Incident date list length: 117914
Complaint date list length: 117914


In [261]:
# Because complaint top location may be slightly lower than items, bisect with a 3px padding added
cr_officer_add = cr_officer_merge.copy()
for cr in cr_action_list:
    cr_idx = bisect.bisect(complaint_keys, (cr['key'] + 3))
    cr_officer_add[cr_idx-1]['action'] = cr['action']
cr_officer_add[:5]

[{'action': '600-No Action Taken',
  'appt_date': '08-Jun-1998',
  'cr_no': '263295',
  'final_category_code': '05A',
  'initial_category_code': '05A',
  'key': 1273,
  'location': 273,
  'officer_name': 'PIWNICKI, RAYMOND',
  'page': 1},
 {'action': '100-Reprimand',
  'appt_date': '08-Jun-1998',
  'cr_no': '263967',
  'final_category_code': '10M',
  'initial_category_code': '10M',
  'key': 1314,
  'location': 314,
  'officer_name': 'PIWNICKI, RAYMOND',
  'page': 1},
 {'action': '600-No Action Taken',
  'appt_date': '08-Jun-1998',
  'cr_no': '265117',
  'final_category_code': '05A',
  'initial_category_code': '05A',
  'key': 1356,
  'location': 356,
  'officer_name': 'PIWNICKI, RAYMOND',
  'page': 1},
 {'action': '600-No Action Taken',
  'appt_date': '08-Jun-1998',
  'cr_no': '266694',
  'final_category_code': '05N',
  'initial_category_code': '05N',
  'key': 1398,
  'location': 398,
  'officer_name': 'PIWNICKI, RAYMOND',
  'page': 1},
 {'action': '600-No Action Taken',
  'appt_date': 

In [262]:
for cr in cr_finding_list:
    cr_idx = bisect.bisect(complaint_keys, (cr['key'] + 3))
    cr_officer_add[cr_idx-1]['finding'] = cr['finding']
cr_officer_add[:5]

[{'action': '600-No Action Taken',
  'appt_date': '08-Jun-1998',
  'cr_no': '263295',
  'final_category_code': '05A',
  'finding': 'NS',
  'initial_category_code': '05A',
  'key': 1273,
  'location': 273,
  'officer_name': 'PIWNICKI, RAYMOND',
  'page': 1},
 {'action': '100-Reprimand',
  'appt_date': '08-Jun-1998',
  'cr_no': '263967',
  'final_category_code': '10M',
  'finding': 'SU',
  'initial_category_code': '10M',
  'key': 1314,
  'location': 314,
  'officer_name': 'PIWNICKI, RAYMOND',
  'page': 1},
 {'action': '600-No Action Taken',
  'appt_date': '08-Jun-1998',
  'cr_no': '265117',
  'final_category_code': '05A',
  'finding': 'NS',
  'initial_category_code': '05A',
  'key': 1356,
  'location': 356,
  'officer_name': 'PIWNICKI, RAYMOND',
  'page': 1},
 {'action': '600-No Action Taken',
  'appt_date': '08-Jun-1998',
  'cr_no': '266694',
  'final_category_code': '05N',
  'finding': 'NS',
  'initial_category_code': '05N',
  'key': 1398,
  'location': 398,
  'officer_name': 'PIWNICKI

In [263]:
for cr in closed_date_list:
    cr_idx = bisect.bisect(complaint_keys, (cr['key'] + 3))
    cr_officer_add[cr_idx-1]['closed_date'] = cr['closed_date']
cr_officer_add[0]

{'action': '600-No Action Taken',
 'appt_date': '08-Jun-1998',
 'closed_date': '05-Jan-2001',
 'cr_no': '263295',
 'final_category_code': '05A',
 'finding': 'NS',
 'initial_category_code': '05A',
 'key': 1273,
 'location': 273,
 'officer_name': 'PIWNICKI, RAYMOND',
 'page': 1}

In [264]:
for cr in incident_date_list:
    cr_idx = bisect.bisect(complaint_keys, (cr['key'] + 3))
    cr_officer_add[cr_idx-1]['incident_date'] = cr['incident_date']
cr_officer_add[0]

{'action': '600-No Action Taken',
 'appt_date': '08-Jun-1998',
 'closed_date': '05-Jan-2001',
 'cr_no': '263295',
 'final_category_code': '05A',
 'finding': 'NS',
 'incident_date': '21 Jun 2000',
 'initial_category_code': '05A',
 'key': 1273,
 'location': 273,
 'officer_name': 'PIWNICKI, RAYMOND',
 'page': 1}

In [265]:
for cr in complaint_date_list:
    cr_idx = bisect.bisect(complaint_keys, (cr['key'] + 3))
    cr_officer_add[cr_idx-1]['complaint_date'] = cr['complaint_date']
cr_officer_add[0]

{'action': '600-No Action Taken',
 'appt_date': '08-Jun-1998',
 'closed_date': '05-Jan-2001',
 'complaint_date': '22 Jun 2000',
 'cr_no': '263295',
 'final_category_code': '05A',
 'finding': 'NS',
 'incident_date': '21 Jun 2000',
 'initial_category_code': '05A',
 'key': 1273,
 'location': 273,
 'officer_name': 'PIWNICKI, RAYMOND',
 'page': 1}

In [266]:
merged_officer_items = pd.DataFrame(cr_officer_add)
merged_officer_items.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117914 entries, 0 to 117913
Data columns (total 13 columns):
action                   113681 non-null object
appt_date                117914 non-null object
closed_date              117785 non-null object
complaint_date           117914 non-null object
cr_no                    117914 non-null object
final_category_code      117914 non-null object
finding                  113637 non-null object
incident_date            117914 non-null object
initial_category_code    117914 non-null object
key                      117914 non-null int64
location                 117914 non-null int64
officer_name             117914 non-null object
page                     117914 non-null int64
dtypes: int64(3), object(10)
memory usage: 11.7+ MB


In [267]:
merged_officer_items.to_csv('officer_complaints_first_draft.csv',index=False)

In [268]:
complaint_category_list = list()
for cr in p_complaint_categories:
    complaint_category_list.append(get_p_loc(cr, 'complaint_category'))
len(complaint_category_list)

235828

In [269]:
# Because complaint top location may be slightly lower than the cr code, bisect with a 3px padding added
# if the difference is greater than 5 px between the heights, add as the final code
for cr in complaint_category_list:
    cr_idx = bisect.bisect(complaint_keys, (cr['key'] + 3))
    # Check if the difference between the code and the complaint row is more than 5 px, if so, add as final
    if ((cr['key'] + 3) - cr_officer_add[cr_idx-1]['key']) > 5:
        cr_officer_add[cr_idx-1]['final_complaint_category'] = cr['complaint_category']
    else:
        cr_officer_add[cr_idx-1]['initial_complaint_category'] = cr['complaint_category']
len(cr_officer_add)

117914

In [270]:
merged_officer_items = pd.DataFrame(cr_officer_add)
merged_officer_items.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117914 entries, 0 to 117913
Data columns (total 15 columns):
action                        113681 non-null object
appt_date                     117914 non-null object
closed_date                   117785 non-null object
complaint_date                117914 non-null object
cr_no                         117914 non-null object
final_category_code           117914 non-null object
final_complaint_category      117914 non-null object
finding                       113637 non-null object
incident_date                 117914 non-null object
initial_category_code         117914 non-null object
initial_complaint_category    117914 non-null object
key                           117914 non-null int64
location                      117914 non-null int64
officer_name                  117914 non-null object
page                          117914 non-null int64
dtypes: int64(3), object(12)
memory usage: 13.5+ MB


In [271]:
merged_officer_items.to_csv('officer_complaints_from_pdf.csv',index=False)